In [83]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, Activation, Dropout
from keras.layers import GlobalMaxPooling2D, MaxPooling2D
from keras.optimizers import Adam
from keras import backend as K

%matplotlib inline

In [97]:
DATA_FILE = 'data/aug_train_data.h5'
METADATA = 'metadata.npy'

f = h5py.File(DATA_FILE, 'r')
data_x = f['x']
data_y = f['y']
metadata = np.load(METADATA).item()

In [98]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
    
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    recall = recall(y_true, y_pred)
    precision = precision(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

In [101]:
lr = 1e-3
n_conv_layers = 3
n_kernels = [128] * n_conv_layers
mp_sizes = [(2,2) for _ in range(n_conv_layers)]

model = Sequential()
model.add(BatchNormalization(axis=3, input_shape=data_x.shape[1:]))
for i in range(n_conv_layers):
    model.add(Conv2D(n_kernels[i], 3, 3, init='he_uniform', border_mode='same'))
    model.add(BatchNormalization(axis=3))
    model.add(Activation('relu'))
    if i < n_conv_layers - 1:
        model.add(MaxPooling2D(mp_sizes[i]))

model.add(Conv2D(8,3,3, border_mode='same'))
model.add(GlobalMaxPooling2D())
model.add(Activation('softmax'))
model.compile(Adam(lr), loss='categorical_crossentropy', metrics=['accuracy', recall, precision, f1])
model.summary()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), kernel_initializer="he_uniform", padding="same")`
  if __name__ == '__main__':


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), padding="same")`
  from ipykernel import kernelapp as app


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_74 (Batc (None, 4, 7, 512)         2048      
_________________________________________________________________
conv2d_70 (Conv2D)           (None, 4, 7, 128)         589952    
_________________________________________________________________
batch_normalization_75 (Batc (None, 4, 7, 128)         512       
_________________________________________________________________
activation_70 (Activation)   (None, 4, 7, 128)         0         
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 2, 3, 128)         0         
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 2, 3, 128)         147584    
_________________________________________________________________
batch_normalization_76 (Batc (None, 2, 3, 128)         512       
__________

In [102]:
from sklearn.model_selection import train_test_split

data_x = np.asarray(data_x)
data_y = np.asarray(data_y)
X_train, X_val, y_train, y_val = train_test_split(data_x, data_y, test_size=0.15, random_state=42)

In [103]:
model.fit(X_train, y_train, batch_size=64, nb_epoch=3,
          validation_data=(X_val, y_val))

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 8694 samples, validate on 1535 samples
Epoch 1/3


  64/8694 [..............................] - ETA: 328s - loss: 2.2242 - acc: 0.0781 - recall: 0.0000e+00 - precision: 0.0000e+00 - f1: nan

 128/8694 [..............................] - ETA: 181s - loss: 2.1800 - acc: 0.1016 - recall: 0.0000e+00 - precision: 0.0000e+00 - f1: nan

 192/8694 [..............................] - ETA: 133s - loss: 2.1360 - acc: 0.1406 - recall: 0.0000e+00 - precision: 0.0000e+00 - f1: nan

...

8448/8694 [============================>.] - ETA: 1s - loss: 0.0460 - acc: 0.9893 - recall: 0.9879 - precision: 0.9913 - f1: 0.9896

8512/8694 [============================>.] - ETA: 0s - loss: 0.0459 - acc: 0.9894 - recall: 0.9880 - precision: 0.9914 - f1: 0.9897

8576/8694 [============================>.] - ETA: 0s - loss: 0.0457 - acc: 0.9895 - recall: 0.9881 - precision: 0.9915 - f1: 0.9898

8640/8694 [============================>.] - ETA: 0s - loss: 0.0456 - acc: 0.9896 - recall: 0.9882 - precision: 0.9915 - f1: 0.9898

8694/8694 [==============================] - 47s - loss: 0.0456 - acc: 0.9895 - recall: 0.9881 - precision: 0.9915 - f1: 0.9898 - val_loss: 0.1944 - val_acc: 0.9401 - val_recall: 0.9309 - val_precision: 0.9514 - val_f1: 0.9410

In [104]:
from sklearn import metrics

TEST_DATA_FILE = 'data/test_data.h5'

tf = h5py.File(TEST_DATA_FILE, 'r')

t_data_x = tf['x']
t_data_y = tf['y']

X_test = np.asarray(t_data_x)
y_test = np.asarray(t_data_y)

preds = model.predict(X_test, batch_size=64, verbose=1)

 64/369 [====>.........................] - ETA: 3s

128/369 [=========>....................] - ETA: 1s

192/369 [==============>...............] - ETA: 0s

256/369 [===================>..........] - ETA: 0s

320/369 [=========================>....] - ETA: 0s

369/369 [==============================] - 1s     

In [105]:
from keras.wrappers.scikit_learn import KerasClassifier

score, acc, recall, precision, f1 = model.evaluate(X_test, y_test, batch_size=64)

print(model.metrics_names)

print('Test score:', score)
print('Test accuracy:', acc)
print('Test precision:', precision)
print('Test recall:', recall)
print('Test F1:', f1)

 64/369 [====>.........................] - ETA: 0s

128/369 [=========>....................] - ETA: 0s

192/369 [==============>...............] - ETA: 0s

256/369 [===================>..........] - ETA: 0s

320/369 [=========================>....] - ETA: 0s

369/369 [==============================] - 0s     


['loss', 'acc', 'recall', 'precision', 'f1']
Test score: 0.178133051782
Test accuracy: 0.945799457995
Test precision: 0.953456361765
Test recall: 0.943089432187
Test F1: 0.948228766117
